# Pattern mining

In [1]:
# https://data-mining.philippe-fournier-viger.com/introduction-to-sequential-rule-mining/

In [2]:
# %pip install awswrangler
# %pip install apyori
# %pip install pyfpgrowth

In [3]:
import os 
import json
import boto3 
import random
#import pyfpgrowth
import numpy as np
import pandas as pd
import awswrangler as wr
from apyori import apriori

from sagemaker import get_execution_role
role = get_execution_role()

In [8]:
selected_region='cosr-nco'
bucket = 'ons-dl-dev-landing'
prefix = 'ons/mpo_ds_nlp/data/postprocessed_output/spacy/sentences/entities-'+selected_region+'.parquet'
input_uri = 's3://{}/{}'.format(bucket, prefix)
df = pd.read_parquet(input_uri)
df

,text,entity,type
0,desenergização da lt 230 kv tucuruí / altamira...,"[desenergização, lt 230 kv tucuruí / altamira]","[ACAO_A_EXECUTAR, EQUIPAMENTO]"
1,procedimentosobjetivo / item de controle1---co...,"[verificar, indisponibilidade, lt 230 kv tucur...","[ACAO_A_EXECUTAR, ESTADO_OPERATIVO, EQUIPAMENTO]"
2,2cosr-ncocosr-ncoequatorialdesligar o religame...,[religamento automático tripolar da lt 230 kv ...,[EQUIPAMENTO]
3,3cosr-ncocosr-ncoeletronorteabrir o terminal d...,"[lt 230 kv tucuruí / altamira, se altamira]","[EQUIPAMENTO, SUBESTACAO]"
4,- reator da lt disponível:se tucuruí ≤ 232 kv-...,"[reator da lt, se tucuruí, ≤, 232 kv-, reator ...","[EQUIPAMENTO, SUBESTACAO, OPERADOR_MATEMATICO,..."
...,...,...,...
3312,estando a transformação 500/230 kv da se sinop...,[em operação],[ESTADO_OPERATIVO]
3313,1cosr-ncocosr-ncocos-state gridenergizar a lt ...,"[lt 500 kv cláudia / sinop, pelo terminal da, ...","[EQUIPAMENTO, ACAO_A_EXECUTAR, SUBESTACAO]"
3314,- transformação 500/230 kv da se sinop desligada.,[desligada],[ACAO_A_EXECUTAR]
3315,- tensão na se sinop superior a 508 kv e ≤ 5...,"[tensão, se sinop, superior a, 508 kv, e ≤, s...","[VALOR_COM_UNID.MEDIDA, SUBESTACAO, OPERADOR_M..."


In [9]:
df = df[df['entity'].map(lambda d: len(d)) > 0]
df

,text,entity,type
0,desenergização da lt 230 kv tucuruí / altamira...,"[desenergização, lt 230 kv tucuruí / altamira]","[ACAO_A_EXECUTAR, EQUIPAMENTO]"
1,procedimentosobjetivo / item de controle1---co...,"[verificar, indisponibilidade, lt 230 kv tucur...","[ACAO_A_EXECUTAR, ESTADO_OPERATIVO, EQUIPAMENTO]"
2,2cosr-ncocosr-ncoequatorialdesligar o religame...,[religamento automático tripolar da lt 230 kv ...,[EQUIPAMENTO]
3,3cosr-ncocosr-ncoeletronorteabrir o terminal d...,"[lt 230 kv tucuruí / altamira, se altamira]","[EQUIPAMENTO, SUBESTACAO]"
4,- reator da lt disponível:se tucuruí ≤ 232 kv-...,"[reator da lt, se tucuruí, ≤, 232 kv-, reator ...","[EQUIPAMENTO, SUBESTACAO, OPERADOR_MATEMATICO,..."
...,...,...,...
3312,estando a transformação 500/230 kv da se sinop...,[em operação],[ESTADO_OPERATIVO]
3313,1cosr-ncocosr-ncocos-state gridenergizar a lt ...,"[lt 500 kv cláudia / sinop, pelo terminal da, ...","[EQUIPAMENTO, ACAO_A_EXECUTAR, SUBESTACAO]"
3314,- transformação 500/230 kv da se sinop desligada.,[desligada],[ACAO_A_EXECUTAR]
3315,- tensão na se sinop superior a 508 kv e ≤ 5...,"[tensão, se sinop, superior a, 508 kv, e ≤, s...","[VALOR_COM_UNID.MEDIDA, SUBESTACAO, OPERADOR_M..."


In [10]:
entity_sequence = df['type'].to_list()
# entity_sequence[0:2]

## Frequent Pattern

In [11]:
# #Finding the frequent patterns with min support threshold=0.5
# FrequentPatterns=pyfpgrowth.find_frequent_patterns(entity_sequence,support_threshold=0.5)
# print(FrequentPatterns)
 
# # Generating rules with min confidence threshold=0.5
# Rules=pyfpgrowth.generate_association_rules(patterns=FrequentPatterns,confidence_threshold=0.5)
# Rules

## RHS and LHS

In [12]:
rules_dict = {
    'rhs':[],
    'lhs':[],
    'support':[],
    'confidence':[],
    'lift':[]
}

rules = apriori(entity_sequence, min_support = 0.05, min_confidence = 0.58, min_lift=1.15, min_length = 2, max_length=3)
rules = list(rules)
rules

# Extracting rules from the object
for i in range(len(rules)):
    rules_dict['rhs'].append(list(rules[i][2][0][0]))
    rules_dict['lhs'].append(list(rules[i][2][0][1]))
    rules_dict['support'].append(rules[i][1])
    rules_dict['confidence'].append(rules[i][2][0][2])
    rules_dict['lift'].append(rules[i][2][0][3])

rules_df = pd.DataFrame(rules_dict)
rules_df.sort_values(by=['lift','confidence'], ascending=False, inplace=True)
rules_df.reset_index(drop=True, inplace=True)
rules_df.head(60)

,rhs,lhs,support,confidence,lift
0,[OPERADOR_MATEMATICO],"[VALOR_COM_UNID.MEDIDA, SUBESTACAO]",0.199879,0.729373,2.784039
1,[OPERADOR_MATEMATICO],[VALOR_COM_UNID.MEDIDA],0.258366,0.942794,2.394524
2,"[OPERADOR_MATEMATICO, EQUIPAMENTO]",[VALOR_COM_UNID.MEDIDA],0.069641,0.895349,2.274022
3,"[ACAO_A_EXECUTAR, SUBESTACAO]",[EQUIPAMENTO],0.182997,0.854930,1.359445
4,"[VALOR_COM_UNID.MEDIDA, ACAO_A_EXECUTAR]",[EQUIPAMENTO],0.056678,0.828194,1.316931
5,[OPERADOR_MATEMATICO],[SUBESTACAO],0.205909,0.751375,1.282053
6,[ACAO_A_EXECUTAR],[EQUIPAMENTO],0.291528,0.793273,1.261403
7,"[OPERADOR_MATEMATICO, EQUIPAMENTO]",[SUBESTACAO],0.056075,0.720930,1.230106


In [13]:
len(rules_df)

8

### Pattern example

In [14]:
df['entity'].iloc[0]

array(['desenergização', 'lt 230 kv tucuruí / altamira'], dtype=object)

## Raw sequence

In [15]:
rules_dict = {
    'sequence':[],
    'support':[],
    'confidence':[],
    'lift':[]
}

association_rules = apriori(entity_sequence, min_support = 0.05, min_confidence = 0.55, min_lift=1.10, min_length = 2, max_length=4)
association_rules = list(association_rules)

for item in association_rules:
    rules_dict['sequence'].append(list(item[0]))
    rules_dict['support'].append(item[1])
    rules_dict['confidence'].append(item[2][0][2])
    rules_dict['lift'].append(item[2][0][3])

sequence_df = pd.DataFrame(rules_dict)
sequence_df.sort_values(by=['lift','support','confidence'], inplace=True, ascending=False)
sequence_df.reset_index(drop=True, inplace=True)
sequence_df.head(20)

,sequence,support,confidence,lift
0,"[VALOR_COM_UNID.MEDIDA, OPERADOR_MATEMATICO, S...",0.199879,0.729373,2.784039
1,"[VALOR_COM_UNID.MEDIDA, OPERADOR_MATEMATICO, S...",0.052759,0.678295,2.589071
2,"[VALOR_COM_UNID.MEDIDA, OPERADOR_MATEMATICO]",0.258366,0.942794,2.394524
3,"[VALOR_COM_UNID.MEDIDA, OPERADOR_MATEMATICO, E...",0.069641,0.895349,2.274022
4,"[ACAO_A_EXECUTAR, SUBESTACAO, EQUIPAMENTO]",0.182997,0.854930,1.359445
5,"[VALOR_COM_UNID.MEDIDA, ACAO_A_EXECUTAR, EQUIP...",0.056678,0.828194,1.316931
6,"[OPERADOR_MATEMATICO, SUBESTACAO]",0.205909,0.751375,1.282053
7,"[ACAO_A_EXECUTAR, EQUIPAMENTO]",0.291528,0.793273,1.261403
8,"[OPERADOR_MATEMATICO, SUBESTACAO, EQUIPAMENTO]",0.056075,0.720930,1.230106
9,"[VALOR_COM_UNID.MEDIDA, SUBESTACAO]",0.261984,0.665391,1.135340


In [17]:
sequence_df['sequence'].iloc[0]

['VALOR_COM_UNID.MEDIDA', 'OPERADOR_MATEMATICO', 'SUBESTACAO']

## Examples

In [26]:
ex_seq = sequence_df['sequence'].iloc[7]
for index, row in df.iterrows():
    if all(item in row['type'] for item in ex_seq):
        print(' text: \n',row['text'])
        print(' entities: \n',row['entity'])
        print(' types: \n',row['type'])
        print('-'*10)

 text: 
 desenergização da lt 230 kv tucuruí / altamira (sentido único passocoordenaçãocontrolecomando, execução.
 entities: 
 ['desenergização' 'lt 230 kv tucuruí / altamira']
 types: 
 ['ACAO_A_EXECUTAR' 'EQUIPAMENTO']
----------
 text: 
 procedimentosobjetivo / item de controle1---cosr-nco---verificar a necessidade de adequações no sistema, para atendimento às limitações impostas pela indisponibilidade lt 230 kv tucuruí / altamira.
 entities: 
 ['verificar' 'indisponibilidade' 'lt 230 kv tucuruí / altamira']
 types: 
 ['ACAO_A_EXECUTAR' 'ESTADO_OPERATIVO' 'EQUIPAMENTO']
----------
 text: 
 - reator da lt disponível:se tucuruí ≤ 232 kv- reator da lt indisponível:se tucuruí ≤ 220 kvdesenergizar a lt 230 kv tucuruí / altamira, pelo terminal da se tucuruí.
 entities: 
 ['reator da lt' 'se tucuruí' '≤' '232 kv-' 'reator da lt' 'se tucuruí' '≤'
 '220 kvdesenergizar' 'lt 230 kv tucuruí / altamira' 'pelo terminal da'
 'se tucuruí']
 types: 
 ['EQUIPAMENTO' 'SUBESTACAO' 'OPERADOR_MATEMATICO'